In [ ]:
import readline

def input_with_prefill(prompt, text):
    def hook():
        readline.insert_text(text)
        readline.redisplay()
    readline.set_pre_input_hook(hook)
    result = input(prompt)
    readline.set_pre_input_hook()
    return result


print(input_with_prefill('Enter something: ', 'default text'))

In [ ]:
import shutil
if shutil.which('pilercr'):
    print('python3 is installed')
else:
    print('python3 is not installed')

In [ ]:
import os

output_file=os.path.join(output_dir, os.path.relpath(mag.rsplit('.', 2 if args.decompress else 1)[0]+"."+tool, input_dir))

tool="minced"
input_dir = '/shares/CIBIO-Storage/CM/scratch/users/isacco.cenacchi/Tirocinio/samples/MAGs_mini'
output_dir = "/shares/CIBIO-Storage/CM/scratch/users/isacco.cenacchi/Tirocinio/out/MAGs_mini_CRISPRDetect3_20241001161255"

# output_file = os.path.join(input_dir.removesuffix(os.path.basename(input_dir)), os.path.basename(args.out))
os.path.basename(input_dir)+'_'+tool+'_parsed.tsv'
os.path.basename(output_dir)+'_parsed.tsv'
os.path.join(output_dir, os.path.basename(input_dir)+'_'+tool+'_parsed.tsv')
# input_dir.removesuffix(os.path.basename(input_dir))

In [ ]:
repeats = ["..A..................................CC.", ".....................................---", "GG............-......................--."]
reference = "AAGTTTCCGTCCCCTTTCGGGGAATCATTTAGAAAAT--A"
line="   145662      33    93.9      33  TAGTATTTAA    ...............T................    AGAAAACCGCATAAGGACCGATACCACATTACA"
stringa = ".--.-...............T................"
count = stringa[:stringa.find(".")].count("-")
seqs = line.split()
start=int(seqs[0]) - seqs[5][:seqs[5].find(".")].count("-")
def develop_repeats(repeats, reference):
    developed_repeats = []
    for repeat in repeats:
        repeat = list(repeat)
        for i in range(len(reference)):
            if repeat[i] == '.': 
                repeat[i] = reference[i]
        repeat = ''.join(repeat)
        repeat = repeat.replace('-', '')
        developed_repeats.append(repeat)
    return developed_repeats

develop_repeats(repeats, reference)
start
# print(develop_repeats(repeats, reference))

In [ ]:
import pandas as pd
from run_CRISPRtools import *

# Path to the uploaded file
gff_file_path = '/Users/isaccocenacchi/Desktop/Tirocinio/tests/M1023330751/M1023330751_real_cpu64_NOdirection.CRISPRDetect3.gff'
# gff_file_path = '/Users/isaccocenacchi/Desktop/Tirocinio/tests/M1023330751/prova.gff'

# Parse the GFF file
crisprs = parse_CRISPRDetect3(gff_file_path)

for crispr in crisprs:
    print(crispr)
    print(crispr.sequence())
    print([len(rep) for rep in crispr.repeats])


In [ ]:
output_root_dir = '/Users/isaccocenacchi/Desktop/Tirocinio/out/MAGs_mini_CRISPRtools'

parsed_files = [os.path.join(dirpath,filename)
                for dirpath, _, filenames in os.walk(output_root_dir)
                for filename in filenames
                if filename.endswith('_parsed.tsv')
            ]

print('\n'.join(parsed_files))

In [ ]:
# LOGGING

%reset -f
import logging
# logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level='INFO')

logger=logging.getLogger(__name__)
logger.setLevel('INFO')

formatter=logging.Formatter('[%(asctime)s] %(levelname)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

handler=logging.StreamHandler()
handler.setLevel('INFO')
handler.setFormatter(formatter)
logger.addHandler(handler)

handler=logging.FileHandler('test.log')
handler.setLevel('INFO')
handler.setFormatter(formatter)
logger.addHandler(handler)

print(f'name: {logger.name}\nhandlers: {logger.handlers}\nparent: {logger.parent}')
logger.info('prova')

for i in range(1,4):
    handler=logging.FileHandler(f'tool{i}.log')
    handler.setLevel('INFO')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    logger.info(f'tool{i}')
    print(f'name: {logger.name}\nhandlers: {logger.handlers}\nparent: {logger.parent}')
    logger.removeHandler(handler)

logger.info('fine')
print(f'name: {logger.name}\nhandlers: {logger.handlers}')
# logger_tool.removeHandler(logging.FileHandler('test.log'))


# logging.shutdown()


import logging
import types

def log_newline(self, how_many_lines=1):
    # Switch handler, output a blank line
    self.removeHandler(self.console_handler)
    self.addHandler(self.blank_handler)
    for i in range(how_many_lines):
        self.info('')

    # Switch back
    self.removeHandler(self.blank_handler)
    self.addHandler(self.console_handler)

def create_logger():
    # Create a handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.DEBUG)
    console_handler.setFormatter(logging.Formatter(fmt="%(name)s %(levelname)-8s: %(message)s"))

    # Create a "blank line" handler
    blank_handler = logging.StreamHandler()
    blank_handler.setLevel(logging.DEBUG)
    blank_handler.setFormatter(logging.Formatter(fmt=''))

    # Create a logger, with the previously-defined handler
    logger = logging.getLogger('logging_test')
    logger.setLevel(logging.DEBUG)
    logger.addHandler(console_handler)

    # Save some data and add a method to logger object
    logger.console_handler = console_handler
    logger.blank_handler = blank_handler
    logger.newline = types.MethodType(log_newline, logger)

    return logger

if __name__ == '__main__':
    logger = create_logger()
    logger.info('Start reading database')
    logger.info('Updating records ...')
    logger.newline()
    logger.info('Finish updating records')


logging.getLoggerClass().newline = log_newline

In [ ]:
# f string indentation
import logging
import types
def x(a):
    return (f'f_name: {a}\n'
                f'contig: {a}\n'
                f'start: {a}\n'
                f'end: {a}\n'
                f'repeats: {a}\n'
                f'spacers: {a}\n'
                )
    
file = 'prova.tsv'
args = types.SimpleNamespace()
args.cas_database = True

logging.error(f'Check the column names in the parsed file {file} '
                f'(MAG, Contig, Start, End, Spacers, Repeats, ToolCodename'
                f'{", Cas_0-1000, Cas_1000-10000, Cas_>100000, Cas_overlayed" if args.cas_database else ""}), '
                f'and secure that file is a TSV file')

   

In [ ]:
from telegram import Bot
from logging import Handler
import logging
import asyncio
import requests
import os

TELEGRAM_BOT_TOKEN=os.environ.get('TELEGRAM_BOT_TOKEN')
TELEGRAM_CHAT_ID=os.environ.get('TELEGRAM_CHAT_ID')

if not TELEGRAM_BOT_TOKEN and not TELEGRAM_CHAT_ID:
     raise Exception('Please set the environment variables TELEGRAM_TOKEN and TELEGRAM_CHAT_ID')


bot = Bot(token=TELEGRAM_BOT_TOKEN)

async def send_message(text, chat_id):
    async with bot:
        await bot.send_message(text=text, chat_id=chat_id)

class BotHandler(Handler):
    def emit(self, record):
        log_entry = self.format(record)
        asyncio.run(send_message(log_entry, TELEGRAM_CHAT_ID))

class RequestsHandler(Handler):
	def emit(self, record):
		log_entry = self.format(record)
		payload = {
			'chat_id': TELEGRAM_CHAT_ID,
			'text': log_entry,
			'parse_mode': 'HTML'
		}
		return requests.post(f'https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage', data=payload).content

logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level='INFO')
logger=logging.getLogger(__name__)

notification_handler=RequestsHandler()
notification_handler.setLevel('ERROR')
# log_formatter=logging.Formatter("<i>[%(asctime)s] %(levelname)s:</i><pre>\n%(message)s</pre>", datefmt='%Y-%m-%d %H:%M:%S')
log_formatter=logging.Formatter('[%(asctime)s] %(levelname)s:\n%(message)s', datefmt='%Y-%m-%d %H:%M:%S')
notification_handler.setFormatter(log_formatter)
logger.addHandler(notification_handler)
 
logger.error('We have a problem')
logger.info('This is an info message')

logger.removeHandler(notification_handler)

In [ ]:
%reset -f
import os
a=os.environ.items()

for key, value in os.environ.items():
    print(f'{key}: {value}')


In [ ]:
repeats=['cccc', "s", 'tttt']
spacers=['aaasf', 'sdf', 'sdf']
spacers1=['','','']

# all(repeat !='' for repeat in repeats)
all(repeats)
# all(spacers)
# [repeat !='' for repeat in repeats]


In [ ]:
columns = {'MAG': str, 'Contig': str, 'Start': int, 'End': int, 'ToolCodename': str, 'Cas_0-1000': int, 'Cas_1000-10000': int, 'Cas_>100000': int, 'Cas_overlayed': int}

columns_groupby = list(columns.keys())
columns_groupby.remove('ToolCodename')

columns_groupby
